### Import dat z excel souborů stažených z valuo do databáze
[valuo - zakoupené cenové údaje](https://profi.valuo.cz/cadastre/prices/purchased)

In [12]:
%load_ext sql
import urllib.parse
from sqlalchemy import create_engine
import pandas as pd
import os

# Zakódování connection stringu
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"
    "Database=VALUO;"
    "Trusted_Connection=yes"
)

# Vytvoření connection stringu
connection_url = "mssql+pyodbc:///?odbc_connect=%s" % params

# Nastavení připojení
%sql $connection_url

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [15]:
%%sql
-- select * from [dbo].[Valuo_data]

select top 10 
		cast(adresa as nvarchar(max)) as adresa
		, lat
		, lon
		, nemovitost
		, plocha
		, mena
		, cenovy_udaj
		, cenovy_udaj/plocha as jc
from valuo_data 
where lat is not null and lon is not null 
		and lat != 0 and lon != 0 
		and nemovitost = 'jednotka' 
		and cenovy_udaj is not null 
		and cenovy_udaj != 0 
		and plocha > 0
		and mena = 'czk'
group by cast(adresa as nvarchar(max)), lat, lon, nemovitost, plocha, mena, cenovy_udaj
order by jc asc

 * mssql+pyodbc:///?odbc_connect=Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BServer%3Dlocalhost%3BDatabase%3DVALUO%3BTrusted_Connection%3Dyes
Done.


adresa,lat,lon,nemovitost,plocha,mena,cenovy_udaj,jc
"Neustupného 1839/8, Stodůlky, 15500 Praha 5",50.0439495,14.3217117,jednotka,43.39,CZK,1.00,0.0230467849734
"Suchý vršek 2099/49, Stodůlky, 15800 Praha 5",50.0483997,14.3440801,jednotka,181.00,CZK,5.00,0.0276243093922
"Suchý vršek 2098/51, Stodůlky, 15800 Praha 5",50.0485065,14.3440570,jednotka,138.00,CZK,5.00,0.0362318840579
"Suchý vršek 2100/47, Stodůlky, 15800 Praha 5",50.0483017,14.3441011,jednotka,138.00,CZK,5.00,0.0362318840579
"Suchý vršek 2097/53, Stodůlky, 15800 Praha 5",50.0486542,14.3439829,jednotka,55.50,CZK,5.00,0.0900900900900
"Suchý vršek 2096/55, Stodůlky, 15800 Praha 5",50.0488121,14.3439208,jednotka,39.10,CZK,5.00,0.1278772378516
"Petržílkova 2565/23, Stodůlky, 15800 Praha 5",50.0505486,14.3425118,jednotka,180.91,CZK,1000.00,5.5276104140180
"Petržílkova 2835/9, Stodůlky, 15800 Praha 5",50.0507102,14.3479341,jednotka,30769.00,CZK,750000.00,24.3751828138711
"Kettnerova 1940/1, Stodůlky, 15500 Praha 5",50.0468593,14.3226874,jednotka,61.00,CZK,1830.00,30.0000000000000
"Fantova 1794/8, Stodůlky, 15500 Praha 5",50.0443151,14.3177839,jednotka,71.00,CZK,3654.00,51.4647887323943


In [14]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
import urllib.parse
from opencage.geocoder import OpenCageGeocode

# API klíč pro získání GPS souřadnic
api_key = '85af71fbd7334627a5b84894066a8a18'
geocoder = OpenCageGeocode(api_key)
processed_addresses = {}

def get_coordinates(address):
    if not address or address == "none":
        return {"LAT": None, "LON": None}
    
    if address in processed_addresses:
        return processed_addresses[address]
    
    try:
        result = geocoder.geocode(address)
        if result and len(result):
            latitude = result[0]['geometry']['lat']
            longitude = result[0]['geometry']['lng']
            processed_addresses[address] = {"LAT": latitude, "LON": longitude}
            print(f"Získány souřadnice: {address} -> ({latitude}, {longitude})")
            return processed_addresses[address]
        else:
            print(f"Nenalezeny souřadnice pro adresu: {address}")
            return {"LAT": None, "LON": None}
    except Exception as e:
        print(f"Chyba při získávání GPS souřadnic pro adresu '{address}': {e}")
        return {"LAT": None, "LON": None}

# Zakódování connection stringu
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"
    "Database=VALUO;"
    "Trusted_Connection=yes"
)

# Vytvoření connection stringu
connection_url = f"mssql+pyodbc:///?odbc_connect={params}"

# Vytvoření SQLAlchemy engine
engine = create_engine(connection_url)

# Cesta ke složce s Excel soubory
directory = r"C:\\Users\\ijttr\\OneDrive\\Dokumenty\\PROG\\PYTHON\\DATA_ANALYSIS\\VALUO\\data"

# Načtení existujících adres s GPS souřadnicemi z databáze
existing_gps_records = pd.read_sql("SELECT adresa, LAT, LON FROM Valuo_data WHERE LAT IS NOT NULL AND LON IS NOT NULL AND LAT != 0 AND LON != 0", engine)
existing_gps_records = existing_gps_records.drop_duplicates(subset=["adresa"], keep="first")  # Odstranění duplicit adres
existing_gps_dict = existing_gps_records.set_index("adresa").to_dict("index")
processed_addresses.update(existing_gps_dict)

# Projít všechny soubory ve složce
for filename in os.listdir(directory):
    if filename.endswith(".xlsx") or filename.endswith(".xls"):
        file_path = os.path.join(directory, filename)
        print(f"Importuji soubor: {filename}")
        
        # Načtení dat z Excelu
        df = pd.read_excel(file_path, sheet_name=0)
        
        # Přemapování názvů sloupců podle SQL tabulky
        df.columns = [
            "cislo_vkladu", "datum_podani", "datum_zplatneni", "listina", "nemovitost", "typ", "adresa", 
            "cenovy_udaj", "mena", "plocha", "typ_plochy", "popis", "okres", "kat_uzemi", "rok", "mesic"
        ]
        
        # Odstranění duplicitních hodnot v rámci jednoho souboru (pouze přesně shodné řádky)
        df.drop_duplicates(keep="first", inplace=True)
        
        # Konverze datových typů
        df["datum_podani"] = pd.to_datetime(df["datum_podani"], errors='coerce')
        df["datum_zplatneni"] = pd.to_datetime(df["datum_zplatneni"], errors='coerce')
        df["cenovy_udaj"] = pd.to_numeric(df["cenovy_udaj"], errors='coerce').fillna(0)
        df["plocha"] = pd.to_numeric(df["plocha"], errors='coerce').fillna(0)
        df["rok"] = df["rok"].astype('Int64').fillna(0)
        df["mesic"] = df["mesic"].astype('Int64').fillna(0)
        
        # Nahrazení všech NULL hodnot v textových sloupcích "none"
        text_columns = ["cislo_vkladu", "listina", "nemovitost", "typ", "adresa", "mena", "typ_plochy", "popis", "okres", "kat_uzemi"]
        df[text_columns] = df[text_columns].fillna("none")
        
        # Kontrola a odstranění existujících záznamů před importem
        existing_records = pd.read_sql("SELECT cislo_vkladu, nemovitost, typ, adresa, plocha, typ_plochy FROM Valuo_data", engine)
        merged = df.merge(existing_records, on=["cislo_vkladu", "nemovitost", "typ", "adresa", "plocha", "typ_plochy"], how="left", indicator=True)
        df = merged[merged["_merge"] == "left_only"].drop(columns=["_merge"], errors='ignore')
        
        if df.empty:
            print(f"Soubor {filename} neobsahuje nové záznamy k importu.")
            print('---------------------------------------------------------------------------------------------')
            continue
        
        # Přidání GPS souřadnic pouze pro budovy a jednotky s platnou adresou
        df["LAT"], df["LON"] = None, None
        mask = (df["nemovitost"].isin(["budova", "jednotka"])) & (df["adresa"] != "none")
        
        # Počet celkových adres k získání GPS
        total_gps_requests = mask.sum()
        
        # Načítání unikátních adres pro minimalizaci API dotazů
        unique_addresses = df.loc[mask, "adresa"].unique()
        unique_gps_requests = sum(1 for addr in unique_addresses if addr not in processed_addresses)
        
        address_coords = {addr: get_coordinates(addr) for addr in unique_addresses if addr not in processed_addresses}
        processed_addresses.update(address_coords)
        
        # Přiřazení GPS souřadnic zpět do tabulky
        df.loc[mask, "LAT"] = df.loc[mask, "adresa"].map(lambda x: processed_addresses.get(x, {"LAT": None, "LON": None})["LAT"])
        df.loc[mask, "LON"] = df.loc[mask, "adresa"].map(lambda x: processed_addresses.get(x, {"LAT": None, "LON": None})["LON"])
        
        # Výpis statistik GPS dotazů
        print(f"GPS CELKEM = {total_gps_requests} / GPS unikátních/získaných = {unique_gps_requests}")
        
        # Vložení dat do SQL tabulky
        df.to_sql("Valuo_data", engine, if_exists='append', index=False)
        print(f"Soubor {filename} byl úspěšně naimportován.")
        print('---------------------------------------------------------------------------------------------')
print("Všechny soubory byly zpracovány.")

Importuji soubor: cadastre_report_10_2023.xlsx
Soubor cadastre_report_10_2023.xlsx neobsahuje nové záznamy k importu.
---------------------------------------------------------------------------------------------
Importuji soubor: cadastre_report_11_2023.xlsx
Soubor cadastre_report_11_2023.xlsx neobsahuje nové záznamy k importu.
---------------------------------------------------------------------------------------------
Importuji soubor: cadastre_report_12_2023.xlsx
Soubor cadastre_report_12_2023.xlsx neobsahuje nové záznamy k importu.
---------------------------------------------------------------------------------------------
Importuji soubor: cadastre_report_1_2024.xlsx
Soubor cadastre_report_1_2024.xlsx neobsahuje nové záznamy k importu.
---------------------------------------------------------------------------------------------
Importuji soubor: cadastre_report_2_2024.xlsx
Soubor cadastre_report_2_2024.xlsx neobsahuje nové záznamy k importu.
--------------------------------------